In [19]:
import pandas as pd
import numpy as np
from IPython.display import display
from IPython.display import HTML
from skspatial.measurement import area_signed
import itertools

# Set display options
pd.options.display.max_columns=None

# Test on 1 game
- The entire map size is 16,000 x 16,000, center at (8000, 8000)

In [50]:
# Read in 1 game
filepath = "./games/lck_spring_2023/"
filename = "ESPORTSTMNT03_3098196.gz"
filegz = filepath + filename
df = pd.read_json(filegz, compression="gzip")

In [68]:
# Split df into 5 min intervals
df["eventTime"] = pd.to_datetime(df["eventTime"])
# df_intervals is array of dataframes in 5min intervals
df_intervals =  [g for n, g in df.groupby(pd.Grouper(key='eventTime',freq='5Min'))]

In [69]:
df_intervals[0]

,eventTime,eventType,platformGameId,participants,stageID,sequenceIndex,gameName,gameVersion,statsUpdateInterval,playbackID,gameTime,spawnTime,monsterName,nextDragonSpawnTime,nextDragonName,gameOver,teams,itemID,participantID,skillSlot,participant,evolved,wardType,position,placer,killer,bountyGold,killType,assistants,monsterType,inEnemyJungle,killerTeamID,level,maxCooldown,summonerSpellName,summonerSpellSlot,victimTeamID,victim,shutdownBounty,killStreakLength,bounty,dragonType,goldGain,itemBeforeUndo,lastHitter,teamID,lane,buildingType,turretTier,itemsAdded,wallTime,winningTeam
0,2023-02-02 08:06:49.492000+00:00,game_info,ESPORTSTMNT03:3098196,"[{'keystoneID': 8008, 'hashedIP': '3VoTUXLOxMy...",1,0,109625152897595358|game1,13.1.487.8994,1000.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-02-02 08:06:49.492000+00:00,queued_epic_monster_info,ESPORTSTMNT03:3098196,NaN,1,1,109625152897595358|game1,NaN,NaN,1,0.0,1200.0,Baron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-02-02 08:06:49.492000+00:00,queued_epic_monster_info,ESPORTSTMNT03:3098196,NaN,1,2,109625152897595358|game1,NaN,NaN,1,0.0,480.0,RiftHerald,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-02-02 08:07:02.337000+00:00,queued_dragon_info,ESPORTSTMNT03:3098196,NaN,1,14,109625152897595358|game1,NaN,NaN,1,164.0,NaN,NaN,300.0,chemtech,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-02-02 08:07:02.337000+00:00,queued_epic_monster_info,ESPORTSTMNT03:3098196,NaN,1,15,109625152897595358|game1,NaN,NaN,1,164.0,300.0,DragonChemtech,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,2023-02-02 08:09:56.220000+00:00,stats_update,ESPORTSTMNT03:3098196,"[{'ability4CooldownRemaining': 0, 'magicPenetr...",1,381,109625152897595358|game1,NaN,NaN,1,155003.0,NaN,NaN,NaN,NaN,0.0,"[{'inhibKills': 0, 'towerKills': 0, 'teamID': ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
240,2023-02-02 08:09:57.045000+00:00,skill_level_up,ESPORTSTMNT03:3098196,NaN,1,383,109625152897595358|game1,NaN,NaN,1,155830.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,6.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
241,2023-02-02 08:09:57.245000+00:00,stats_update,ESPORTSTMNT03:3098196,"[{'ability4CooldownRemaining': 0, 'magicPenetr...",1,385,109625152897595358|game1,NaN,NaN,1,156028.0,NaN,NaN,NaN,NaN,0.0,"[{'inhibKills': 0, 'towerKills': 0, 'teamID': ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
242,2023-02-02 08:09:58.237000+00:00,stats_update,ESPORTSTMNT03:3098196,"[{'ability4CooldownRemaining': 0, 'magicPenetr...",1,386,109625152897595358|game1,NaN,NaN,1,157020.0,NaN,NaN,NaN,NaN,0.0,"[{'inhibKills': 0, 'towerKills': 0, 'teamID': ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Getting player positions

In [44]:
# Test on 1 file
part_df = df["participants"].dropna().reset_index(drop=True)
# NEED TO ITERATE OVER ALL ROWS
part_df_row = part_df.iloc[1929]

blue_players_filter = filter(lambda x: x["teamID"] == 100, part_df_row)
blue_players = list(blue_players_filter)
blue_positions_map = map(lambda p: (p["position"]["x"], p["position"]["z"]), blue_players)
blue_positions = list(blue_positions_map)
blue_positions

[(4000, 602), (2139, 677), (642, 287), (396, 520), (3050, 656)]

In [47]:
# The entire map size is 16,000 x 16,000, center at (8000, 8000)

In [46]:
def calculate_area(point1, point2, point3):
    x1, y1 = point1
    x2, y2 = point2
    x3, y3 = point3
    
    area = 0.5 * (x1 * (y2 - y3) + x2 * (y3 - y1) + x3 * (y1 - y2))
    return area

# Define your set of points
points = [(2,4), (3, -6), (7,8)]

# Generate all combinations of 3 points
combinations = itertools.combinations(blue_positions, 3)

triangle_count = 0
for combo in combinations:
    p1, p2, p3 = combo
    # Check if the points are not collinear (non-zero area)
    if calculate_area(p1, p2, p3) > 0:
        triangle_count += 1
        area = calculate_area(p1, p2, p3)
        print(f"Triangle {triangle_count}: Points {p1}, {p2}, {p3}, Area = {area}")

print(f"Total number of triangles: {triangle_count}")

Triangle 1: Points (4000, 602), (2139, 677), (642, 287), Area = 419032.5
Triangle 2: Points (4000, 602), (2139, 677), (396, 520), Area = 211451.0
Triangle 3: Points (2139, 677), (642, 287), (3050, 656), Area = 193363.5
Triangle 4: Points (2139, 677), (396, 520), (3050, 656), Area = 89815.0
Total number of triangles: 4
